In [ ]:
!pip install geopy
import geopandas as gpd
import geocoder
import json
import pandas as pd
from geopy.geocoders import Nominatim

from shapely.geometry import LineString, Point, Polygon



In [ ]:
import geopandas as gpd
import geocoder
import json
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

from google.colab import files
uploaded = files.upload()
#file_name = list(uploaded.keys())[0]
#df = pd.read_csv(file_name)
#print(df)



In [ ]:
import pandas as pd

LOCAL_FILE_PATH = '/content/'
LOCAL_FILE_NAME = '婦幼安全警示地點-1121005.csv'
# 使用pandas讀取本地CSV檔案
df = pd.read_csv(LOCAL_FILE_PATH + LOCAL_FILE_NAME, sep=',')

# 將數據儲存為新的CSV檔案
df.to_csv('new_file.csv', index=False, encoding='UTF-8')


df.fillna(0, inplace=True)
print(df)

df.columns = df.iloc[0]


df = df[1:]

print(df)


      No                Address    DeptNm  BranchNm Contact  ContactNumber
0     編號                   地點位置     管轄警察局        分局    聯繫窗口           聯繫電話
1      1                  捷運中山站  臺北市政府警察局      大同分局     葉警員  (02)2553-4657
2      2    峨嵋街、漢中街口(捷運西門站6號出口)  臺北市政府警察局      萬華分局     羅警員  (02)2375-3990
3      3  西園路、和平西路口(捷運龍山寺站1號出口)  臺北市政府警察局      萬華分局     羅警員  (02)2375-3990
4      4                捷運中山國小站  臺北市政府警察局      中山分局     顏警員  (02)2551-5495
..   ...                    ...       ...       ...     ...            ...
239  239               臺鐵新城車站公廁     鐵路警察局      花蓮分局    曾偵查佐   (03)833-2310
240  240              臺鐵羅東車站手扶梯     鐵路警察局      花蓮分局    曾偵查佐   (03)833-2310
241  241               臺鐵頭城車站電梯     鐵路警察局      花蓮分局    曾偵查佐   (03)833-2310
242  242              臺南科學園區迎曦湖  保安警察第二總隊  第一大隊第二中隊     陳警員   (06)505-5672
243  243  高雄市前鎮區過港隧道旗津往前鎮機車道出口處  高雄港務警察總隊      中興中隊     逄警員   (07)571-0483

[244 rows x 6 columns]
0     編號                   地點位置     管轄警察局        分局  聯繫窗口           聯繫電話
1  

In [ ]:
# 清理 "地點位置" 列的值，刪除 "一帶"
df['地點位置'] = df['地點位置'].str.replace('一帶', '')

# 創建測試數據
data = {'地點位置': ['桃園市中壢區文化二路', '桃園市中壢區莒光路', '桃園市中壢區中正路'],
        '管轄警察局': ['桃園市政府警察局', '桃園市政府警察局', '桃園市政府警察局'],
        '分局': ['中壢分局', '中壢分局', '中壢分局'],
        '聯繫窗口': ['李家防官', '李家防官', '李家防官'],
        '聯繫電話': ['(03)422-4918', '(03)422-4918', '(03)422-4918']}

df = pd.DataFrame(data)

# 從原始數據框創建副本
df_loc = df[df['分局'] == '中壢分局'].copy()

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# 定義函數，將地址轉經緯度
def address_to_coordinates(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None
    except GeocoderUnavailable:
        print("Geocoder service is unavailable. Retrying...")
        return None

# 將"地點位置"列的地址信息轉換為經緯度
df_loc['經緯度'] = df_loc['地點位置'].apply(address_to_coordinates)

# 檢查經緯度坐標是否為空，將空值填充為 (None, None)
df_loc['經緯度'] = df_loc['經緯度'].apply(lambda x: (None, None) if x is None else x)

# 拆分經緯度信息為兩列
df_loc[['經度', '緯度']] = pd.DataFrame(df_loc['經緯度'].tolist(), index=df_loc.index)

# 顯示結果
print(df_loc)

df_loc.to_csv('new.csv', index=False, encoding='UTF-8')


         地點位置     管轄警察局    分局  聯繫窗口          聯繫電話                        經緯度  \
0  桃園市中壢區文化二路  桃園市政府警察局  中壢分局  李家防官  (03)422-4918   (24.9772666, 121.250848)   
1   桃園市中壢區莒光路  桃園市政府警察局  中壢分局  李家防官  (03)422-4918  (24.9516317, 121.2362138)   
2   桃園市中壢區中正路  桃園市政府警察局  中壢分局  李家防官  (03)422-4918  (24.9715112, 121.2007311)   

          經度          緯度  
0  24.977267  121.250848  
1  24.951632  121.236214  
2  24.971511  121.200731  
